In [22]:
import pandas as pd
import numpy as np
import os
from urllib.request import urlretrieve
import geopandas as gpd

In [ ]:
# Load in tables of data itself and not the summary sheets
df = pd.read_excel('../../data/landing/projections/VIF2023_projections.xlsx', sheet_name=['Component_Detail_Annual', '5yr_Age_Groups_Sex_2011_2051'], header=None)

In [3]:
population_changes = df['Component_Detail_Annual']
indv_changes = df['5yr_Age_Groups_Sex_2011_2051']

In [ ]:
# Here we index to only grab the table data and it's content in the excel sheets
population_changes = population_changes[9:50]
# Rename each column to accruately represent the table data
population_changes.columns = ["Year", "Population at Start of Period", "Births", "Deaths", "Natural Increase", "Overseas Migration Arrivals", "Overseas Migration Departures", "Net Overseas Migration", "Interstate Migration Arrivals", "Interstate Migration Departures", "Net Interstate Migration", "Net Migration", "Population at End of Period", "Change in Population", "Annual Population Growth Rate", "Statistical Difference*"]

In [5]:
#We'll grab only useful columns
population_changes = population_changes[["Year", "Net Overseas Migration", "Net Interstate Migration", "Population at End of Period", "Change in Population"]]
population_changes

,Year,Net Overseas Migration,Net Interstate Migration,Population at End of Period,Change in Population
9,2011,NaN,NaN,5537817,NaN
10,2012,56174,2417,5651091,113274
11,2013,59034,6420,5772669,121578
12,2014,56906,9739,5894917,122248
13,2015,60694,11079,6022322,127405
14,2016,72215,17639,6173172,150850
15,2017,91243,13748,6302608,129436
16,2018,86968,9101,6423038,120430
17,2019,85476,6480,6537305,114267
18,2020,60597,-2652,6615046,77741


In [ ]:
# Rename columns to match the table data
indv_changes.columns = indv_changes.iloc[8]
# Makes sure we only read in data
indv_changes = indv_changes[9:]

In [ ]:
# Here we change the dataframe columns so we have year and population as two seperate variables
indv_changes = indv_changes.melt(
    id_vars=["Sex", "Age", "5-year Age/Sex group", "15-year Age/Sex group"],
    var_name="Year",
    value_name="Population"
)

In [8]:
indv_changes

,Sex,Age,5-year Age/Sex group,15-year Age/Sex group,Year,Population
0,Males,0 to 4,Males - 0 to 4,Males - 0 to 14,2011.0,1.808130e+05
1,Males,5 to 9,Males - 5 to 9,Males - 0 to 14,2011.0,1.704800e+05
2,Males,10 to 14,Males - 10 to 14,Males - 0 to 14,2011.0,1.694040e+05
3,Males,15 to 19,Males - 15 to 19,Males - 15 to 29,2011.0,1.816580e+05
4,Males,20 to 24,Males - 20 to 24,Males - 15 to 29,2011.0,2.110180e+05
...,...,...,...,...,...,...
2373,Persons,75 to 79,Persons - 75 to 79,Persons - 75 and over,2051.0,3.653008e+05
2374,Persons,80 to 84,Persons - 80 to 84,Persons - 75 and over,2051.0,3.127423e+05
2375,Persons,85 to 89,Persons - 85 to 89,Persons - 75 and over,2051.0,2.237413e+05
2376,Persons,90 and over,Persons - 90 and over,Persons - 75 and over,2051.0,1.734436e+05


In [ ]:
# Now we get the total popualtion for each 15 year age group
indv_changes = (
    indv_changes
    .groupby(["15-year Age/Sex group", "Year"], as_index=False)["Population"]
    .sum()
)

In [10]:
indv_changes

,15-year Age/Sex group,Year,Population
0,Females - 0 to 14,2011.0,4.940390e+05
1,Females - 0 to 14,2012.0,5.043300e+05
2,Females - 0 to 14,2013.0,5.161920e+05
3,Females - 0 to 14,2014.0,5.275550e+05
4,Females - 0 to 14,2015.0,5.387120e+05
...,...,...,...
774,Persons - Total,2047.0,9.822517e+06
775,Persons - Total,2048.0,9.949489e+06
776,Persons - Total,2049.0,1.007612e+07
777,Persons - Total,2050.0,1.020240e+07


In [ ]:
# We'll now make each 15-age groups it's own variable with it's corresponding year
indv_changes = indv_changes.pivot(
    index="Year",
    columns="15-year Age/Sex group",
    values="Population"
).reset_index()

In [ ]:
# We already have Total population from first dataframe
indv_changes = indv_changes.drop(columns=["Persons - Total"])

In [ ]:
# Final merged data frame of both tables
population_data = pd.merge(population_changes, indv_changes, on="Year", how="left")

In [ ]:
# We'll remove 2011 data due to missing values and limited relevance given it's age
population_data = population_data.drop(population_data.index[0])
# Round all numbers down
population_data = np.floor(population_data).astype(int)

In [15]:
# List columns for females, males, and persons
female_cols = [col for col in population_data.columns if col.startswith("Females")]
male_cols = [col for col in population_data.columns if col.startswith("Males")]
person_cols = [col for col in population_data.columns if col.startswith("Persons")]

# Convert to percentages of total population
population_data[female_cols] = population_data[female_cols].div(population_data["Population at End of Period"], axis=0) 
population_data[male_cols] = population_data[male_cols].div(population_data["Population at End of Period"], axis=0) 
population_data[person_cols] = population_data[person_cols].div(population_data["Population at End of Period"], axis=0) 

# Round to 2 decimals
population_data[female_cols + male_cols + person_cols] = population_data[female_cols + male_cols + person_cols].round(2)


In [16]:
#Rename columns
population_data.rename(columns={'Population at End of Period': 'Population'}, inplace=True)

#This code was made with assitance of Chat GPT to efficeint rename columns
population_data.rename(columns=lambda x: f'Percentage - {x}' if x.startswith(('Females', 'Males', 'Persons')) else x, inplace=True)

In [17]:
population_data

,Year,Net Overseas Migration,Net Interstate Migration,Population,Change in Population,Percentage - Females - 0 to 14,Percentage - Females - 15 to 29,Percentage - Females - 30 to 44,Percentage - Females - 45 to 59,Percentage - Females - 60 to 74,...,Percentage - Males - 30 to 44,Percentage - Males - 45 to 59,Percentage - Males - 60 to 74,Percentage - Males - 75 and over,Percentage - Persons - 0 to 14,Percentage - Persons - 15 to 29,Percentage - Persons - 30 to 44,Percentage - Persons - 45 to 59,Percentage - Persons - 60 to 74,Percentage - Persons - 75 and over
1,2012,56174,2417,5651091,113274,0.09,0.11,0.11,0.10,0.07,...,0.11,0.09,0.06,0.03,0.18,0.21,0.21,0.19,0.13,0.07
2,2013,59034,6420,5772669,121578,0.09,0.10,0.11,0.10,0.07,...,0.11,0.09,0.06,0.03,0.18,0.21,0.21,0.19,0.13,0.07
3,2014,56906,9739,5894917,122248,0.09,0.10,0.11,0.10,0.07,...,0.11,0.09,0.07,0.03,0.18,0.21,0.21,0.19,0.13,0.07
4,2015,60694,11079,6022322,127405,0.09,0.10,0.11,0.10,0.07,...,0.11,0.09,0.07,0.03,0.18,0.21,0.21,0.19,0.13,0.07
5,2016,72215,17639,6173172,150850,0.09,0.10,0.11,0.10,0.07,...,0.11,0.09,0.07,0.03,0.18,0.21,0.21,0.19,0.14,0.07
6,2017,91243,13748,6302608,129436,0.09,0.10,0.11,0.10,0.07,...,0.11,0.09,0.07,0.03,0.18,0.21,0.21,0.19,0.14,0.07
7,2018,86968,9101,6423038,120430,0.09,0.10,0.11,0.10,0.07,...,0.11,0.09,0.07,0.03,0.18,0.21,0.21,0.19,0.14,0.07
8,2019,85476,6480,6537305,114267,0.09,0.10,0.11,0.09,0.07,...,0.11,0.09,0.07,0.03,0.18,0.21,0.21,0.18,0.14,0.07
9,2020,60597,-2652,6615046,77741,0.09,0.10,0.11,0.09,0.07,...,0.11,0.09,0.07,0.03,0.18,0.20,0.22,0.18,0.14,0.07
10,2021,-53059,-35622,6547822,-67224,0.09,0.10,0.11,0.09,0.08,...,0.11,0.09,0.07,0.03,0.18,0.19,0.22,0.19,0.15,0.07


In [ ]:
# Create the processed data directory if it doesn't exist
os.makedirs('../../data/processed/population', exist_ok=True)

# Save the processed population data to a new CSV file
population_data.to_csv('../../data/processed/population/population_data.csv', index=False)

In [ ]:
# Load in Alternative population dataset
df2 = pd.read_excel('../../data/landing/projections/sa2_population_hist.xlsx', sheet_name=['Table 1'], header=None)

In [ ]:
location_population = df2['Table 1']
# Rename columns to match the table data
location_population.columns = ["GCCSA code",
    "GCCSA name",
    "SA4 code",
    "SA4 name",
    "SA3 code",
    "SA3 name",
    "SA2 code",
    "SA2 name",
    "2001",
    "2002",
    "2003",
    "2004",
    "2005",
    "2006",
    "2007",
    "2008",
    "2009",
    "2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
    "2022",
    "2023",
]
#Makes sure we only read in data
location_population = location_population[7:2461]

In [25]:
#Filter for only VIC population data
vic_data = location_population[
    location_population["GCCSA name"].isin(["Greater Melbourne", "Rest of Vic."])
]

In [ ]:
# Checking we only have VIC data
vic_data["GCCSA name"].unique()

array(['Rest of Vic.', 'Greater Melbourne'], dtype=object)

In [ ]:
# Check for NUll values
vic_data.isnull().values.any()

False

In [ ]:
# Convert year columns into rows
vic_data = vic_data.melt(
    id_vars=[
        "GCCSA code", "GCCSA name",
        "SA4 code", "SA4 name", "SA3 code", "SA3 name",
        "SA2 code", "SA2 name"
    ],
    var_name="Year",
    value_name="Population"
)

In [29]:
vic_data

,GCCSA code,GCCSA name,SA4 code,SA4 name,SA3 code,SA3 name,SA2 code,SA2 name,Year,Population
0,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,201011001,Alfredton,2001,5756
1,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,201011002,Ballarat,2001,11497
2,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,201011005,Buninyong,2001,5320
3,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,201011006,Delacombe,2001,4154
4,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,201011007,Smythes Creek,2001,3317
...,...,...,...,...,...,...,...,...,...,...
12001,2RVIC,Rest of Vic.,217,Warrnambool and South West,21703,Colac - Corangamite,217031476,Otway,2023,3983
12002,2RVIC,Rest of Vic.,217,Warrnambool and South West,21704,Warrnambool,217041477,Moyne - East,2023,7132
12003,2RVIC,Rest of Vic.,217,Warrnambool and South West,21704,Warrnambool,217041478,Moyne - West,2023,10148
12004,2RVIC,Rest of Vic.,217,Warrnambool and South West,21704,Warrnambool,217041479,Warrnambool - North,2023,22762


In [ ]:
# Only keep useful variables
vic_data = vic_data[["SA2 code", "SA2 name", "Year", "Population"]]

In [31]:
vic_data

,SA2 code,SA2 name,Year,Population
0,201011001,Alfredton,2001,5756
1,201011002,Ballarat,2001,11497
2,201011005,Buninyong,2001,5320
3,201011006,Delacombe,2001,4154
4,201011007,Smythes Creek,2001,3317
...,...,...,...,...
12001,217031476,Otway,2023,3983
12002,217041477,Moyne - East,2023,7132
12003,217041478,Moyne - West,2023,10148
12004,217041479,Warrnambool - North,2023,22762


In [32]:
# Load shapefile
shp_path = "../../data/landing/boundaries/SA2_2021_AUST_GDA2020.shp"
sa2 = gpd.read_file(shp_path)

# Rename 
sa2 = sa2.rename(columns={"SA2_CODE21": "SA2 code"})

sa2 = sa2[["SA2 code", "geometry"]]

# Ensure SA2 codes are string with zero padding
sa2["SA2 code"] = sa2["SA2 code"].astype(str).str.zfill(9)
vic_data["SA2 code"] = vic_data["SA2 code"].astype(str).str.zfill(9)

# Merge with population data
gdf = sa2.merge(vic_data, on="SA2 code", how="inner")

C:\Users\bachk\AppData\Local\Temp\ipykernel_10404\292424956.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vic_data["SA2 code"] = vic_data["SA2 code"].astype(str).str.zfill(9)


In [33]:
gdf

,SA2 code,geometry,SA2 name,Year,Population
0,201011001,"POLYGON ((143.78282 -37.56666, 143.75558 -37.5...",Alfredton,2001,5756
1,201011001,"POLYGON ((143.78282 -37.56666, 143.75558 -37.5...",Alfredton,2002,6092
2,201011001,"POLYGON ((143.78282 -37.56666, 143.75558 -37.5...",Alfredton,2003,6293
3,201011001,"POLYGON ((143.78282 -37.56666, 143.75558 -37.5...",Alfredton,2004,6480
4,201011001,"POLYGON ((143.78282 -37.56666, 143.75558 -37.5...",Alfredton,2005,6648
...,...,...,...,...,...
12001,217041480,"POLYGON ((142.45281 -38.39126, 142.4523 -38.39...",Warrnambool - South,2019,13264
12002,217041480,"POLYGON ((142.45281 -38.39126, 142.4523 -38.39...",Warrnambool - South,2020,13346
12003,217041480,"POLYGON ((142.45281 -38.39126, 142.4523 -38.39...",Warrnambool - South,2021,13298
12004,217041480,"POLYGON ((142.45281 -38.39126, 142.4523 -38.39...",Warrnambool - South,2022,13323


In [34]:
map = pd.read_csv('../../data/landing/boundaries/sa2_to_postcode_mapping.csv')
map = map.rename(columns={"SA2_CODE_2021": "SA2 code", "POSTCODE": "Postcode"})

In [ ]:
# Filter for only VIC population data
map_vic_data = map[
    map["state"].isin(["VIC"])
]

In [ ]:
# Clean up SA2 code column to match other dataframes
map_vic_data = map_vic_data.dropna(subset=["SA2 code"])
map_vic_data["SA2 code"] = map_vic_data["SA2 code"].round().astype(int)

In [37]:
# Merge with postcode data
gdf["SA2 code"] = gdf["SA2 code"].astype(str).str.zfill(9)
map_vic_data["SA2 code"] = map_vic_data["SA2 code"].astype(str).str.zfill(9)
final = gdf.merge(map_vic_data, on="SA2 code", how="inner")

In [38]:
population_final = final[["SA2 code", "postcode", "Year", "Population"]]

In [39]:
population_final = population_final.drop_duplicates()
population_final = population_final[population_final['Year'] == "2023"]
population_final = population_final.drop(columns=["Year"])
population_final = population_final.rename(columns={"Population": "Population-2023"})
population_final

,SA2 code,postcode,Population-2023
66,201011005,3357,7323
91,201011006,3358,12869
180,201011008,3355,15431
756,201011484,3350,12885
779,201011484,3356,12885
...,...,...,...
79645,217041478,3301,10148
79660,217041478,3304,10148
79760,217041479,3281,22762
80006,217041480,3277,13476


In [ ]:
# Create the processed data directory if it doesn't exist
os.makedirs('../../data/processed/population', exist_ok=True)

# Save the processed population location data to a new CSV file
population_final.to_csv('../../data/processed/population/population_final.csv', index=False)